In [ ]:
import copy

from transformers import AutoProcessor, AutoModelForCausalLM
from PIL import Image
import requests

import utils
import os
import torch

%matplotlib inline

In [ ]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
model_id = 'microsoft/Florence-2-large'
model = AutoModelForCausalLM.from_pretrained(model_id, trust_remote_code=True).eval().cuda()
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True)
utils.set_model_info(model, processor)

In [ ]:
path = "image.png"
image = Image.open(path)
image_rgb = Image.open(path).convert("RGB")

In [ ]:
image

In [ ]:
task = utils.TaskType.OPEN_VOCAB_DETECTION

test_labels = ['rubbish bin', 'street light', 'building', 'electricity pole']
for label in test_labels:
    results = utils.run_example(task, image_rgb, text_input = label)
    bbox_results  = utils.convert_to_od_format(results[utils.TaskType.OPEN_VOCAB_DETECTION])
    #print(bbox_results)
    utils.plot_bbox(bbox_results, image)